In [16]:
%load_ext autoreload
%autoreload 2
from glob import glob
import os
import re
import sys
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
paths_feim = glob('../output/feature_importances/20190907*.gz') \
+ glob('../output/feature_importances/20190909*.gz') \
+ glob('../output/feature_importances/20190908*.gz')
list_good_feature = []

for path in paths_feim:
    try:
        lb_score = re.search(rf'CV([^/.]*)__', path).group(1)
    except AttributeError:
        pass
    if float(lb_score.replace('-', '.'))<0.943:
        continue
    feim = read_pkl_gzip(path)
    tmp_list = feim[feim['imp_avg']>10000].index.tolist()
#     tmp_list = feim[(feim['imp_avg']<2000) & (feim['imp_avg']>100)].index.tolist()
    for feature in tmp_list:
        if feature.count('ProductCD-'):
            if feature in list_good_feature:
                continue
            list_good_feature.append(feature)

In [57]:
sorted(list_good_feature)

['502__user_id_card_addr_pemail_M__C13__ProductCD-W_max_min_diff',
 '502__user_id_card_addr_pemail__C1__ProductCD-C_max_min_diff',
 '505__user_id_card_addr_pemail_day3_TransactionAmt__ProductCD-S_sum_org_ratio',
 '505__user_id_card_addr_pemail_day3_TransactionAmt__ProductCD-W_sum_org_diff',
 '505__user_id_card_addr_pemail_day3_TransactionAmt__ProductCD-W_sum_org_ratio',
 '505__user_id_card_addr_pemail_day5_TransactionAmt__ProductCD-C_sum_org_diff',
 '505__user_id_card_addr_pemail_day5_TransactionAmt__ProductCD-W_mean_org_diff',
 '506__user_id_card_addr_pemail__C10__ProductCD-C__diff_last-first__C2__diff_last-first',
 '506__user_id_card_addr_pemail__C10__ProductCD-C__ratio_last-first__C10__ratio_last-first',
 '506__user_id_card_addr_pemail__C10__diff_last-first__C12__ProductCD-C__ratio_last-first',
 '506__user_id_card_addr_pemail__C10__diff_last-first__C13__ProductCD-W__diff_last-first',
 '506__user_id_card_addr_pemail__C10__ratio_last-first__C13__ProductCD-C__ratio_last-first',
 '506__

In [56]:
import shutil

# list_from_dir = ['raw_use' , 'org_use']
# list_from_dir += ['raw_trush' , 'org_trush', 'valid_trush']
# from_dir = 'raw_trush'
from_dir = 'valid_trush'
from_dir = 'product_feature'
to_dir = 'valid_use'
list_not_found = []

for feature in list_good_feature:
#     if not feature.count('V'):
#         continue
    path_train_feature = f"../feature/{from_dir}/{feature}_train.gz"
    path_test_feature = f"../feature/{from_dir}/{feature}_test.gz"

    if os.path.exists(path_train_feature) and os.path.exists(path_test_feature):
        try:
            shutil.move(path_train_feature, f'../feature/{to_dir}')
            shutil.move(path_test_feature, f'../feature/{to_dir}')
        except FileNotFoundError:
            print(feature)
            list_not_found.append(feature)